This file is produced by Hengyi Li and Jingbo Wang

2023.3.10

# Fundamentals of machine learning

## Generalization: The goal of machine learning

We can setup our envrinment by importing those libraries 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import imdb

2023-03-10 11:26:45.010431: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Before start, we can check our running environment by running those code

In [2]:
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_physical_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Improving generalization

To improve the generalization, we have several ways to do it. 

* Dataset curation

* Feature engineering

* Using early stopping

* Reducing the network's size

  * Reducing the network's size
  * Adding L2 weight regularization to the model
  * Adding dropout to the IMDB model

### Dataset curation

* Make sure you have enough data. Remember that you need a _**dense sampling**_ of the input-cross-output space. More data will yield a better model. Sometimes, problems that seem impossible at first become solvable with a larger dataset.
* Make sure you have enough data. Remember that you need a dense sampling of the input-cross-output space. More data will yield a better model. Sometimes, problems that seem impossible at first become solvable with a larger dataset.
* Clean your data and deal with missing values.
* If you have many features and you aren’t sure which ones are actually useful, do feature selection.


So the num_words restricted the top 10,000 most frequently occurring words in the training data. Rare words will be discarded. And we can do better than that by increasing the num_words to 25000 

**Original Model**

In [3]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 5s 114ms/step - loss: 0.5311 - accuracy: 0.7761 - val_loss: 0.4004 - val_accuracy: 0.8680
Epoch 2/20
30/30 [==============================] - 1s 22ms/step - loss: 0.3317 - accuracy: 0.8913 - val_loss: 0.3254 - val_accuracy: 0.8796
Epoch 3/20
30/30 [==============================] - 1s 18ms/step - loss: 0.2471 - accuracy: 0.9183 - val_loss: 0.2850 - val_accuracy: 0.8882
Epoch 4/20
30/30 [==============================] - 1s 18ms/step - loss: 0.2020 - accuracy: 0.9322 - val_loss: 0.2822 - val_accuracy: 0.8847
Epoch 5/20
30/30 [==============================] - 1s 20ms/step - loss: 0.1672 - accuracy: 0.9450 - val_loss: 0.2838 - val_accuracy: 0.8860
Epoch 6/20
30/30 [==============================] - 1s 18ms/step - loss: 0.1402 - accuracy: 0.9557 - val_loss: 0.2840 - val_accuracy: 0.8892
Epoch 7/20
30/30 [==============================] - 0s 16ms/step - loss: 0.1222 - accuracy: 0.9609 - val_loss: 0.2960 - val_accuracy: 0.8854
Epoch 8/20
3

**Practice Model**

Here, we change the num_words to 25000 

In [4]:
(train_data, train_labels), _ = imdb.load_data(num_words=25000)

def vectorize_sequences(sequences, dimension=25000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 10s 293ms/step - loss: 0.5482 - accuracy: 0.7693 - val_loss: 0.4233 - val_accuracy: 0.8596
Epoch 2/20
30/30 [==============================] - 3s 110ms/step - loss: 0.3386 - accuracy: 0.9008 - val_loss: 0.3370 - val_accuracy: 0.8749
Epoch 3/20
30/30 [==============================] - 2s 58ms/step - loss: 0.2389 - accuracy: 0.9309 - val_loss: 0.2837 - val_accuracy: 0.8949
Epoch 4/20
30/30 [==============================] - 2s 83ms/step - loss: 0.1806 - accuracy: 0.9473 - val_loss: 0.2708 - val_accuracy: 0.8951
Epoch 5/20
30/30 [==============================] - 2s 70ms/step - loss: 0.1425 - accuracy: 0.9605 - val_loss: 0.2758 - val_accuracy: 0.8905
Epoch 6/20
30/30 [==============================] - 2s 81ms/step - loss: 0.1133 - accuracy: 0.9697 - val_loss: 0.2874 - val_accuracy: 0.8863
Epoch 7/20
30/30 [==============================] - 3s 86ms/step - loss: 0.0910 - accuracy: 0.9780 - val_loss: 0.2869 - val_accuracy: 0.8892
Epoch 8/20

As it shown above, after change the num_words and the batch size, make a bigger dataset and more training on each epoch, the accuracy gets visible improvement

### Feature engineering

**Original Model**

In [17]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 4s 96ms/step - loss: 0.5205 - accuracy: 0.7763 - val_loss: 0.4033 - val_accuracy: 0.8504
Epoch 2/50
30/30 [==============================] - 1s 22ms/step - loss: 0.3199 - accuracy: 0.8953 - val_loss: 0.3093 - val_accuracy: 0.8846
Epoch 3/50
30/30 [==============================] - 1s 18ms/step - loss: 0.2401 - accuracy: 0.9223 - val_loss: 0.2886 - val_accuracy: 0.8835
Epoch 4/50
30/30 [==============================] - 1s 17ms/step - loss: 0.1958 - accuracy: 0.9355 - val_loss: 0.2817 - val_accuracy: 0.8844
Epoch 5/50
30/30 [==============================] - 0s 16ms/step - loss: 0.1651 - accuracy: 0.9451 - val_loss: 0.2761 - val_accuracy: 0.8898
Epoch 6/50
30/30 [==============================] - 0s 16ms/step - loss: 0.1378 - accuracy: 0.9573 - val_loss: 0.2863 - val_accuracy: 0.8864
Epoch 7/50
30/30 [==============================] - 1s 19ms/step - loss: 0.1176 - accuracy: 0.9639 - val_loss: 0.3380 - val_accuracy: 0.8733
Epoch 8/50
30

**Practice Model**

We can enhance by deleting the most common words

In [18]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)
common_stop_words=["a","an","and","the","in","of","to","that","this","for"]


def delete_common_stop_words(train_data, common_stop_words):
    word_index = imdb.get_word_index()
    for i in range(len(common_stop_words)):
      if word_index[common_stop_words[i]]:
         common_stop_word_index = word_index[common_stop_words[i]]
         for i in range(len(train_data)):
           train_data[i] = list(
               filter((common_stop_word_index).__ne__, train_data[i]))
    return train_data

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = delete_common_stop_words(train_data, common_stop_words)
train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
Practice_Run  = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 4s 89ms/step - loss: 0.5674 - accuracy: 0.7741 - val_loss: 0.4500 - val_accuracy: 0.8478
Epoch 2/50
30/30 [==============================] - 1s 25ms/step - loss: 0.3725 - accuracy: 0.8817 - val_loss: 0.3425 - val_accuracy: 0.8779
Epoch 3/50
30/30 [==============================] - 0s 16ms/step - loss: 0.2766 - accuracy: 0.9097 - val_loss: 0.3019 - val_accuracy: 0.8851
Epoch 4/50
30/30 [==============================] - 0s 16ms/step - loss: 0.2214 - accuracy: 0.9248 - val_loss: 0.2793 - val_accuracy: 0.8894
Epoch 5/50
30/30 [==============================] - 0s 16ms/step - loss: 0.1861 - accuracy: 0.9372 - val_loss: 0.2756 - val_accuracy: 0.8894
Epoch 6/50
30/30 [==============================] - 0s 16ms/step - loss: 0.1570 - accuracy: 0.9496 - val_loss: 0.3016 - val_accuracy: 0.8791
Epoch 7/50
30/30 [==============================] - 1s 18ms/step - loss: 0.1366 - accuracy: 0.9563 - val_loss: 0.2865 - val_accuracy: 0.8895
Epoch 8/50
30

### Using early stopping

We could using early stopping class in the Keras library to accomplish this step

**Original model**

In [7]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 4s 107ms/step - loss: 0.5482 - accuracy: 0.7753 - val_loss: 0.4280 - val_accuracy: 0.8564
Epoch 2/50
30/30 [==============================] - 1s 20ms/step - loss: 0.3526 - accuracy: 0.8903 - val_loss: 0.3417 - val_accuracy: 0.8721
Epoch 3/50
30/30 [==============================] - 1s 17ms/step - loss: 0.2660 - accuracy: 0.9113 - val_loss: 0.2912 - val_accuracy: 0.8885
Epoch 4/50
30/30 [==============================] - 0s 16ms/step - loss: 0.2124 - accuracy: 0.9295 - val_loss: 0.2768 - val_accuracy: 0.8905
Epoch 5/50
30/30 [==============================] - 1s 20ms/step - loss: 0.1787 - accuracy: 0.9400 - val_loss: 0.2749 - val_accuracy: 0.8915
Epoch 6/50
30/30 [==============================] - 1s 18ms/step - loss: 0.1535 - accuracy: 0.9489 - val_loss: 0.2807 - val_accuracy: 0.8889
Epoch 7/50
30/30 [==============================] - 1s 17ms/step - loss: 0.1322 - accuracy: 0.9582 - val_loss: 0.2899 - val_accuracy: 0.8857
Epoch 8/50
3

So we could apply the early stopping method as shown below

**Practice**

In [8]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=0.9,
    restore_best_weights=True,
    start_from_epoch=0,
)
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])


model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_practice = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4, callbacks=[callback])

Epoch 1/50
30/30 [==============================] - 4s 98ms/step - loss: 0.5318 - accuracy: 0.7745 - val_loss: 0.4031 - val_accuracy: 0.8708
Epoch 2/50
30/30 [==============================] - 1s 24ms/step - loss: 0.3321 - accuracy: 0.8949 - val_loss: 0.3148 - val_accuracy: 0.8853
Epoch 3/50
30/30 [==============================] - 1s 19ms/step - loss: 0.2491 - accuracy: 0.9191 - val_loss: 0.2881 - val_accuracy: 0.8892
Epoch 4/50
30/30 [==============================] - 1s 24ms/step - loss: 0.2014 - accuracy: 0.9333 - val_loss: 0.2794 - val_accuracy: 0.8861
Epoch 5/50
30/30 [==============================] - 1s 40ms/step - loss: 0.1718 - accuracy: 0.9417 - val_loss: 0.2826 - val_accuracy: 0.8880
Epoch 6/50
30/30 [==============================] - 1s 30ms/step - loss: 0.1458 - accuracy: 0.9534 - val_loss: 0.2972 - val_accuracy: 0.8818
Epoch 7/50
30/30 [==============================] - 1s 25ms/step - loss: 0.1280 - accuracy: 0.9597 - val_loss: 0.2926 - val_accuracy: 0.8866
Epoch 8/50
30

In [9]:
len(history_practice.history['loss'])

36

From the experiments above we could saw that the early stopping take action in the epoch 36 with the accuracy of 0.9999. This is our best results so far and it does save a lot of computing resources

### Regularizing your model

#### Reducing the network's size

**Original model**

In [10]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 5s 129ms/step - loss: 0.5029 - accuracy: 0.7909 - val_loss: 0.3745 - val_accuracy: 0.8713
Epoch 2/20
30/30 [==============================] - 1s 21ms/step - loss: 0.3081 - accuracy: 0.8973 - val_loss: 0.3498 - val_accuracy: 0.8563
Epoch 3/20
30/30 [==============================] - 1s 17ms/step - loss: 0.2362 - accuracy: 0.9197 - val_loss: 0.3088 - val_accuracy: 0.8731
Epoch 4/20
30/30 [==============================] - 1s 19ms/step - loss: 0.1896 - accuracy: 0.9372 - val_loss: 0.2731 - val_accuracy: 0.8925
Epoch 5/20
30/30 [==============================] - 1s 17ms/step - loss: 0.1582 - accuracy: 0.9483 - val_loss: 0.2909 - val_accuracy: 0.8843
Epoch 6/20
30/30 [==============================] - 1s 17ms/step - loss: 0.1283 - accuracy: 0.9607 - val_loss: 0.2891 - val_accuracy: 0.8878
Epoch 7/20
30/30 [==============================] - 1s 22ms/step - loss: 0.1120 - accuracy: 0.9662 - val_loss: 0.3810 - val_accuracy: 0.8616
Epoch 8/20
3

**Version of the model with lower capacity**

In [11]:
model = keras.Sequential([
    layers.Dense(4, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_smaller_model = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 5s 106ms/step - loss: 0.6189 - accuracy: 0.6869 - val_loss: 0.5715 - val_accuracy: 0.8308
Epoch 2/20
30/30 [==============================] - 1s 19ms/step - loss: 0.5222 - accuracy: 0.8052 - val_loss: 0.5194 - val_accuracy: 0.7895
Epoch 3/20
30/30 [==============================] - 1s 17ms/step - loss: 0.4732 - accuracy: 0.8582 - val_loss: 0.4981 - val_accuracy: 0.8174
Epoch 4/20
30/30 [==============================] - 0s 16ms/step - loss: 0.4401 - accuracy: 0.8899 - val_loss: 0.4872 - val_accuracy: 0.8344
Epoch 5/20
30/30 [==============================] - 0s 16ms/step - loss: 0.4155 - accuracy: 0.9095 - val_loss: 0.4675 - val_accuracy: 0.8732
Epoch 6/20
30/30 [==============================] - 0s 16ms/step - loss: 0.3950 - accuracy: 0.9256 - val_loss: 0.4748 - val_accuracy: 0.8543
Epoch 7/20
30/30 [==============================] - 0s 15ms/step - loss: 0.3775 - accuracy: 0.9365 - val_loss: 0.4548 - val_accuracy: 0.8822
Epoch 8/20
3

**Version of the model with higher capacity**

In [12]:
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(512, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_larger_model = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 7s 189ms/step - loss: 0.5576 - accuracy: 0.7157 - val_loss: 0.3372 - val_accuracy: 0.8601
Epoch 2/20
30/30 [==============================] - 4s 120ms/step - loss: 0.3475 - accuracy: 0.8525 - val_loss: 0.2842 - val_accuracy: 0.8827
Epoch 3/20
30/30 [==============================] - 3s 115ms/step - loss: 0.2331 - accuracy: 0.9107 - val_loss: 0.2703 - val_accuracy: 0.8890
Epoch 4/20
30/30 [==============================] - 3s 101ms/step - loss: 0.1917 - accuracy: 0.9269 - val_loss: 0.2931 - val_accuracy: 0.8824
Epoch 5/20
30/30 [==============================] - 3s 102ms/step - loss: 0.1480 - accuracy: 0.9417 - val_loss: 0.3265 - val_accuracy: 0.8841
Epoch 6/20
30/30 [==============================] - 3s 105ms/step - loss: 0.1173 - accuracy: 0.9556 - val_loss: 0.2855 - val_accuracy: 0.8841
Epoch 7/20
30/30 [==============================] - 3s 102ms/step - loss: 0.0781 - accuracy: 0.9747 - val_loss: 0.3480 - val_accuracy: 0.8861
Epoch 

#### Adding weight regularization

**Adding L2 weight regularization to the model**

In [13]:
model = keras.Sequential([
    layers.Dense(16,
                 kernel_regularizer=regularizers.l2(0.002),
                 activation="relu"),
    layers.Dense(16,
                 kernel_regularizer=regularizers.l2(0.002),
                 activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_l2_reg = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 4s 103ms/step - loss: 0.6129 - accuracy: 0.7816 - val_loss: 0.4863 - val_accuracy: 0.8632
Epoch 2/20
30/30 [==============================] - 1s 21ms/step - loss: 0.4162 - accuracy: 0.8891 - val_loss: 0.4059 - val_accuracy: 0.8768
Epoch 3/20
30/30 [==============================] - 1s 19ms/step - loss: 0.3380 - accuracy: 0.9101 - val_loss: 0.3739 - val_accuracy: 0.8804
Epoch 4/20
30/30 [==============================] - 1s 18ms/step - loss: 0.2973 - accuracy: 0.9261 - val_loss: 0.3842 - val_accuracy: 0.8720
Epoch 5/20
30/30 [==============================] - 1s 19ms/step - loss: 0.2737 - accuracy: 0.9321 - val_loss: 0.3572 - val_accuracy: 0.8860
Epoch 6/20
30/30 [==============================] - 1s 18ms/step - loss: 0.2562 - accuracy: 0.9388 - val_loss: 0.3548 - val_accuracy: 0.8861
Epoch 7/20
30/30 [==============================] - 1s 17ms/step - loss: 0.2460 - accuracy: 0.9419 - val_loss: 0.3700 - val_accuracy: 0.8812
Epoch 8/20
3

**Different weight regularizers available in Keras**

In [14]:

regularizers.l1(0.001)
regularizers.l1_l2(l1=0.001, l2=0.001)

#### Adding dropout

**Adding dropout to the IMDB model**

In [15]:
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_dropout = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 4s 96ms/step - loss: 0.6421 - accuracy: 0.6239 - val_loss: 0.5276 - val_accuracy: 0.8383
Epoch 2/20
30/30 [==============================] - 1s 21ms/step - loss: 0.5221 - accuracy: 0.7589 - val_loss: 0.4350 - val_accuracy: 0.8562
Epoch 3/20
30/30 [==============================] - 1s 18ms/step - loss: 0.4385 - accuracy: 0.8156 - val_loss: 0.3325 - val_accuracy: 0.8810
Epoch 4/20
30/30 [==============================] - 1s 17ms/step - loss: 0.3725 - accuracy: 0.8556 - val_loss: 0.3051 - val_accuracy: 0.8863
Epoch 5/20
30/30 [==============================] - 1s 17ms/step - loss: 0.3152 - accuracy: 0.8835 - val_loss: 0.2801 - val_accuracy: 0.8894
Epoch 6/20
30/30 [==============================] - 1s 17ms/step - loss: 0.2779 - accuracy: 0.9026 - val_loss: 0.2760 - val_accuracy: 0.8919
Epoch 7/20
30/30 [==============================] - 1s 17ms/step - loss: 0.2451 - accuracy: 0.9155 - val_loss: 0.2757 - val_accuracy: 0.8894
Epoch 8/20
30

## Summary